In [1]:
import numpy as np
import pandas as pd
from scipy.stats import pearsonr
import os
import re

#获取数据
def load(path): 
    matrix=pd.read_table(path+'/matrix',sep=' ',header=None)
    spectra=pd.read_table(path+'/spectra',header=None)
    spectra.columns=['name']
    matrix.rename(columns={matrix.columns[-1]:'error'},inplace=True)
    matrix.loc[matrix.error=='+','error']=0
    matrix.loc[matrix.error=='-','error']=1
    #映射loc
    ids,seen,count=list(),set(),9
    for i,row in spectra.iterrows():
        id=row['name'].split('#')[-1]
        name=row['name'].replace(id, '')
        if name not in seen:
            seen.add(name)
            count+=1
        ids.append(int(id)+count*100000)

    matrix.columns=ids+['error']

    return matrix


#特征选择
def feature_select(data):
    data_x=data.iloc[:,:-1]
    data_y=data.iloc[:,-1]
    data_x.loc[:, (data_x != 0).any(axis=0)]
    delete=[]
    for i,x in data_x.iteritems():
        if len(x)<2:continue
        rela=pearsonr(x,data_y)[0]
        if rela<=0:
            delete.append(x.name)      
    for d in delete:
        data.drop(d, axis=1,inplace=True)
            
    return data
            

#遍历各项目版本文件
def travel_file(path):
    programs=['Chart','Lang','Math','Time']
    address=list()
    for prog in programs:
        for root,dirs,files in os.walk(path+'/'+prog):
            for dir in dirs:
                
                if re.match(r'[0-9]',dir):
                    address.append(path+'/'+prog+'/'+dir)
    return address


                
    
    




In [2]:
load('./defects4j/Chart/1')

,1000087,1000088,1000089,1000091,1000092,1000093,1000103,1000114,1100049,1200072,...,6100098,6100099,6100101,6100110,6100121,6100122,6100124,6100125,6100127,error
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,1,0,1,1,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,1,0,1,1,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
423,1,1,0,1,1,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
424,1,1,0,1,1,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
425,1,1,0,1,1,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
426,1,1,0,1,1,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0


In [23]:
#生成降维覆盖矩阵文件
for add in travel_file('./defects4j'):
    matrix=load(add)
    
    matrix=feature_select(matrix)
    matrix.to_excel(add+'/matrix.xlsx')
    


D:\Conda\lib\site-packages\scipy\stats\stats.py:3845: PearsonRConstantInputWarning: An input array is constant; the correlation coefficent is not defined.
  warnings.warn(PearsonRConstantInputWarning())


KeyError: '[2400009] not found in axis'